In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("📊 Análisis Exploratorio de Datos - Mercado Automotriz US")
print("=" * 60)

# ================================
# 1. CARGA Y EXPLORACIÓN INICIAL
# ================================

print("\n1️⃣ CARGA Y EXPLORACIÓN INICIAL DEL DATASET")
print("-" * 50)

# Cargar datos
df = pd.read_csv('../data/vehicles_us.csv')

print(f"✅ Dataset cargado exitosamente!")
print(f"📏 Dimensiones: {df.shape[0]:,} filas x {df.shape[1]} columnas")
print(f"💾 Tamaño en memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Información general
print(f"\n📋 INFORMACIÓN GENERAL:")
print(f"Columnas: {list(df.columns)}")
print(f"\nTipos de datos:")
print(df.dtypes)

# Valores faltantes
print(f"\n❌ VALORES FALTANTES:")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Columna': missing_data.index,
    'Valores Faltantes': missing_data.values,
    'Porcentaje': missing_percent.values
}).sort_values('Porcentaje', ascending=False)

print(missing_df[missing_df['Valores Faltantes'] > 0])

# ================================
# 2. LIMPIEZA DE DATOS
# ================================

print(f"\n2️⃣ LIMPIEZA Y PREPROCESAMIENTO")
print("-" * 50)

# Antes de la limpieza
print(f"Registros antes de limpieza: {len(df):,}")

# Eliminar valores faltantes críticos
df_clean = df.dropna(subset=['price', 'model_year']).copy()
print(f"Después de eliminar NaN en price/model_year: {len(df_clean):,}")

# Filtrar valores extremos
df_clean = df_clean[
    (df_clean['price'] > 0) & 
    (df_clean['price'] < 100000) &
    (df_clean['model_year'] >= 1990)
]
print(f"Después de filtrar valores extremos: {len(df_clean):,}")

# Conversiones de tipos
df_clean['date_posted'] = pd.to_datetime(df_clean['date_posted'], errors='coerce')
df_clean['model_year'] = df_clean['model_year'].astype(int)

# Completar valores faltantes en categóricas
categorical_cols = ['condition', 'fuel', 'transmission', 'type', 'paint_color']
for col in categorical_cols:
    df_clean[col] = df_clean[col].fillna('unknown')

# Variables derivadas
current_year = datetime.now().year
df_clean['age'] = current_year - df_clean['model_year']
df_clean['price_category'] = pd.cut(df_clean['price'], 
                                   bins=[0, 5000, 15000, 30000, 50000, float('inf')],
                                   labels=['Muy Bajo', 'Bajo', 'Medio', 'Alto', 'Muy Alto'])

print(f"✅ Dataset limpio final: {len(df_clean):,} registros")
print(f"📊 Datos eliminados: {len(df) - len(df_clean):,} ({((len(df) - len(df_clean))/len(df)*100):.1f}%)")

# ================================
# 3. ESTADÍSTICAS DESCRIPTIVAS
# ================================

print(f"\n3️⃣ ESTADÍSTICAS DESCRIPTIVAS")
print("-" * 50)

# Variables numéricas
numeric_cols = ['price', 'model_year', 'odometer', 'cylinders', 'days_listed', 'age']
print("📈 VARIABLES NUMÉRICAS:")
print(df_clean[numeric_cols].describe().round(2))

# Variables categóricas
print(f"\n📊 VARIABLES CATEGÓRICAS:")
for col in categorical_cols:
    print(f"\n{col.upper()}:")
    value_counts = df_clean[col].value_counts().head(5)
    total = len(df_clean)
    for value, count in value_counts.items():
        percentage = (count/total) * 100
        print(f"  {value}: {count:,} ({percentage:.1f}%)")

# ================================
# 4. ANÁLISIS DE PRECIOS
# ================================

print(f"\n4️⃣ ANÁLISIS DETALLADO DE PRECIOS")
print("-" * 50)

price_stats = {
    'Promedio': df_clean['price'].mean(),
    'Mediana': df_clean['price'].median(),
    'Mínimo': df_clean['price'].min(),
    'Máximo': df_clean['price'].max(),
    'Desviación Estándar': df_clean['price'].std(),
    'Q1 (25%)': df_clean['price'].quantile(0.25),
    'Q3 (75%)': df_clean['price'].quantile(0.75)
}

print("💰 ESTADÍSTICAS DE PRECIOS:")
for stat, value in price_stats.items():
    print(f"  {stat}: ${value:,.0f}")

# Análisis por categorías
print(f"\n💎 PRECIO PROMEDIO POR CATEGORÍAS:")

# Por tipo de combustible
fuel_prices = df_clean.groupby('fuel')['price'].agg(['mean', 'median', 'count']).round(0)
print(f"\nPor Combustible:")
print(fuel_prices.sort_values('mean', ascending=False))

# Por condición
condition_prices = df_clean.groupby('condition')['price'].agg(['mean', 'median', 'count']).round(0)
print(f"\nPor Condición:")
print(condition_prices.sort_values('mean', ascending=False))

# Por tipo de vehículo
type_prices = df_clean.groupby('type')['price'].agg(['mean', 'median', 'count']).round(0)
print(f"\nPor Tipo de Vehículo:")
print(type_prices.sort_values('mean', ascending=False))

# ================================
# 5. ANÁLISIS DE CORRELACIONES
# ================================

print(f"\n5️⃣ ANÁLISIS DE CORRELACIONES")
print("-" * 50)

# Matriz de correlación
correlation_matrix = df_clean[numeric_cols].corr()
print("🔗 MATRIZ DE CORRELACIÓN:")
print(correlation_matrix.round(3))

# Correlaciones más fuertes con el precio
price_corr = correlation_matrix['price'].abs().sort_values(ascending=False)
print(f"\n📊 CORRELACIONES MÁS FUERTES CON EL PRECIO:")
for var, corr in price_corr.items():
    if var != 'price':
        direction = "positiva" if correlation_matrix.loc['price', var] > 0 else "negativa"
        print(f"  {var}: {correlation_matrix.loc['price', var]:.3f} ({direction})")

# ================================
# 6. ANÁLISIS DE MODELOS POPULARES
# ================================

print(f"\n6️⃣ ANÁLISIS DE MODELOS Y MARCAS")
print("-" * 50)

# Top 10 modelos más populares
top_models = df_clean['model'].value_counts().head(10)
print("🚗 TOP 10 MODELOS MÁS POPULARES:")
total_vehicles = len(df_clean)
for model, count in top_models.items():
    percentage = (count / total_vehicles) * 100
    avg_price = df_clean[df_clean['model'] == model]['price'].mean()
    print(f"  {model}: {count:,} vehículos ({percentage:.1f}%) - Precio promedio: ${avg_price:,.0f}")

# Análisis de valor por dinero
print(f"\n💡 ANÁLISIS DE VALOR POR DINERO (Top 10 modelos):")
value_analysis = df_clean[df_clean['model'].isin(top_models.index)].groupby('model').agg({
    'price': 'mean',
    'model_year': 'mean',
    'odometer': 'mean'
}).round(0)

value_analysis['price_per_year'] = value_analysis['price'] / (current_year - value_analysis['model_year'] + 1)
value_analysis_sorted = value_analysis.sort_values('price_per_year')

print(value_analysis_sorted)

# ================================
# 7. ANÁLISIS TEMPORAL
# ================================

print(f"\n7️⃣ ANÁLISIS TEMPORAL")
print("-" * 50)

if df_clean['date_posted'].notna().any():
    # Análisis por mes
    df_clean['post_month'] = df_clean['date_posted'].dt.to_period('M')
    monthly_stats = df_clean.groupby('post_month').agg({
        'price': ['mean', 'count'],
        'days_listed': 'mean'
    }).round(2)
    
    print("📅 ESTADÍSTICAS MENSUALES (últimos 6 meses disponibles):")
    print(monthly_stats.tail(6))
    
    # Análisis de velocidad de venta
    avg_days_listed = df_clean['days_listed'].mean()
    print(f"\n⏱️ VELOCIDAD DE VENTA:")
    print(f"  Promedio días en el mercado: {avg_days_listed:.1f} días")
    
    # Por rango de precio
    days_by_price = df_clean.groupby('price_category')['days_listed'].mean()
    print(f"\n  Días promedio por rango de precio:")
    for category, days in days_by_price.items():
        print(f"    {category}: {days:.1f} días")

# ================================
# 8. INSIGHTS Y CONCLUSIONES
# ================================

print(f"\n8️⃣ INSIGHTS Y CONCLUSIONES PRINCIPALES")
print("-" * 50)

# Calcular insights automáticos
insights = []

# Modelo más popular
most_popular = df_clean['model'].mode()[0]
most_popular_count = df_clean['model'].value_counts().iloc[0]
insights.append(f"El modelo más popular es {most_popular} con {most_popular_count:,} vehículos")

# Combustible más caro
fuel_avg_prices = df_clean.groupby('fuel')['price'].mean()
most_expensive_fuel = fuel_avg_prices.idxmax()
insights.append(f"Los vehículos de {most_expensive_fuel} tienen el precio promedio más alto (${fuel_avg_prices.max():,.0f})")

# Correlación precio-kilometraje
price_mileage_corr = df_clean['price'].corr(df_clean['odometer'])
if abs(price_mileage_corr) > 0.3:
    direction = "negativa" if price_mileage_corr < 0 else "positiva"
    insights.append(f"Correlación {direction} significativa entre precio y kilometraje ({price_mileage_corr:.3f})")

# Tracción 4WD
if 'is_4wd' in df_clean.columns:
    pct_4wd = (df_clean['is_4wd'].sum() / len(df_clean)) * 100
    avg_price_4wd = df_clean[df_clean['is_4wd'] == 1]['price'].mean()
    avg_price_2wd = df_clean[df_clean['is_4wd'] == 0]['price'].mean()
    premium_4wd = ((avg_price_4wd - avg_price_2wd) / avg_price_2wd) * 100
    insights.append(f"{pct_4wd:.1f}% de vehículos tienen 4WD, con un premium promedio de {premium_4wd:.1f}%")

# Depreciación
oldest_cars = df_clean[df_clean['age'] >= 10]
newest_cars = df_clean[df_clean['age'] <= 3]
if len(oldest_cars) > 0 and len(newest_cars) > 0:
    depreciation = ((newest_cars['price'].mean() - oldest_cars['price'].mean()) / newest_cars['price'].mean()) * 100
    insights.append(f"Los vehículos pierden aproximadamente {depreciation:.1f}% de su valor en 10+ años")

print("🔍 INSIGHTS PRINCIPALES:")
for i, insight in enumerate(insights, 1):
    print(f"  {i}. {insight}")

# ================================
# 9. RECOMENDACIONES
# ================================

print(f"\n9️⃣ RECOMENDACIONES PARA COMPRADORES/VENDEDORES")
print("-" * 50)

print("💡 PARA COMPRADORES:")
print("  • Buscar vehículos con 3-7 años para mejor relación precio-valor")
print("  • Considerar vehículos con alto kilometraje para mayor ahorro")
print(f"  • El modelo {most_popular} ofrece buena disponibilidad y valor de reventa")
print("  • Evitar vehículos con más de 150,000 millas si se busca durabilidad")

print(f"\n💰 PARA VENDEDORES:")
print("  • Vehículos más nuevos (< 3 años) mantienen mejor valor")
print("  • Condición 'excellent' puede justificar precio premium")
print(f"  • Combustible {most_expensive_fuel} tiene mejor valor de mercado")
print("  • Considerar timing de venta basado en análisis temporal")

print(f"\n✅ ANÁLISIS COMPLETADO")
print("=" * 60)
print("📊 Para visualizaciones interactivas, ejecutar: streamlit run app.py")
print("📈 Para análisis adicional, modificar este notebook según necesidades específicas")

# ================================
# 10. CÓDIGO PARA VISUALIZACIONES 
# ================================

# Descomentar las siguientes líneas para generar gráficos en Jupyter


# Configurar matplotlib para mostrar gráficos
%matplotlib inline

# 1. Distribución de precios
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(df_clean['price'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('Distribución de Precios')
plt.xlabel('Precio ($)')
plt.ylabel('Frecuencia')
plt.ticklabel_format(style='plain', axis='x')

plt.subplot(1, 2, 2)
plt.boxplot(df_clean['price'])
plt.title('Box Plot de Precios')
plt.ylabel('Precio ($)')
plt.ticklabel_format(style='plain', axis='y')

plt.tight_layout()
plt.show()

# 2. Top 10 modelos
plt.figure(figsize=(12, 6))
top_models.plot(kind='barh', color='lightcoral')
plt.title('Top 10 Modelos Más Populares')
plt.xlabel('Cantidad de Vehículos')
plt.tight_layout()
plt.show()

# 3. Precio vs Kilometraje
plt.figure(figsize=(10, 6))
plt.scatter(df_clean['odometer'], df_clean['price'], alpha=0.5, s=1)
plt.title('Precio vs Kilometraje')
plt.xlabel('Kilometraje')
plt.ylabel('Precio ($)')
plt.ticklabel_format(style='plain')
plt.show()

# 4. Matriz de correlación
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlación')
plt.tight_layout()
plt.show()

# 5. Precio promedio por combustible
plt.figure(figsize=(10, 6))
fuel_prices['mean'].plot(kind='bar', color='lightgreen')
plt.title('Precio Promedio por Tipo de Combustible')
plt.ylabel('Precio Promedio ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'plotly'